# Finger People experimental playground

In [30]:
import numpy as np
import cv2
import os
import math
from helpers import imshow


## Input Sources
### Video file

In [20]:
cap = cv2.VideoCapture('samples/all_moves_1_360.mp4') 

# all_moves_1 sizes: 360, 180, 90, 46

### Camera

In [21]:
cap = cv2.VideoCapture(0)

# reduce frame size to speed it up
w = 180
cap.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 


True

## Optical Flow of Hand
#### Skin Detection and Optical Flow

In [4]:
# HSV ranges for skin detection  
skin_hsv_lower = np.array([0,  48,  80 ], dtype = "uint8")
skin_hsv_upper = np.array([20, 255, 255], dtype = "uint8")

def hsv_mask(img, lower, upper):
    '''
    Given a BGR image, returns a mask of pixels within lower and upper HSV space
    
    mask = hsv_mask(img, lower, upper)
    
    input
        img: A BGR image
        lower: 3-tuple with hue, saturation, and value of lower cutoff of mask
        upper: 3-tuple with hue, saturation, and value of upper cutoff of mask
        
    output
        mask: binary mask of pixels with HSV values between lower and upper
    '''
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    
    return mask


In [41]:
cap = cv2.VideoCapture('samples/all_moves_1_180.mp4') 

MIN_BLOB_SIZE = 200

# Parameters for farneback optical flow
fb_params = dict( pyr_scale = 0.5, 
                  levels = 3, 
                  winsize = 5, 
                  iterations = 3, 
                  poly_n = 5,
                  poly_sigma = 1.2, 
                  flags = 0 )

# Take the first frame
_ret, init_frame = cap.read()
prvs = cv2.cvtColor(init_frame,cv2.COLOR_BGR2GRAY)
frame1 = cv2.flip(init_frame, 1)

# For color representation of optical flow
flow_vis = np.zeros_like(init_frame)
flow_vis[...,1] = 255

while(cap.isOpened()):
    
    # Get the next frame
    _ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  
    if not _ret:
        break
        
    ''' SKIN MASK '''
    skin_mask = hsv_mask(frame, skin_hsv_lower, skin_hsv_upper)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel, iterations = 2)
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel, iterations = 2)
    
    im, contours, hierarchy = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    lowest_contour = None
    lowest_contour_height = math.inf
    for i, contour in enumerate(contours):
        
        if cv2.contourArea(contour) >= MIN_BLOB_SIZE:
            
            # Find the lowest contour (that should be the hand, not face)
            m = cv2.moments(contour)
            contour_height = m['m10']/m['m00']
            
            if contour_height < lowest_contour_height:
                lowest_contour = i
                lowest_contour_height = contour_height
            
    # Draw in hand blob
    hand_mask = np.zeros_like(skin_mask)
    if lowest_contour:
        cv2.drawContours(hand_mask, contours, lowest_contour, 255, -1)

    ''' OPTICAL FLOW '''
    next = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, **fb_params)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    # Ignore non-hand movement
    mag = cv2.bitwise_and(mag, mag, mask=hand_mask)    
    
    ''' MHI '''

    ''' DEBUG/OUTPUT '''
    # Make skin mask same shape as frame
    hand_mask = cv2.cvtColor(hand_mask, cv2.COLOR_GRAY2BGR)
    
    # Create color representation of optical flow
    flow_vis[...,0] = ang*180/np.pi/2
    flow_vis[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    flow_vis_bgr = cv2.cvtColor(flow_vis,cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame',np.hstack((frame, hand_mask, flow_vis_bgr)))

    # Exit on ESC
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    prvs = next
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1